## Table of contents:
1. Cleaning table
 * translate column names to English 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
url = 'https://raw.githubusercontent.com/mborycki/Silesian_Real_Estate_Market/master'
olx_link = f'{url}/OLX_Offers.csv'
location_list = f'{url}/locations.txt'

In [ ]:
olx_df = pd.read_csv(olx_link,thousands=',')

In [ ]:
olx_df.columns

In [ ]:
olx_df.shape

In [ ]:
olx_df.head()

## Data Cleaning:

### Translate column names to English:

In [ ]:
olx_df.rename(columns={'Cena': 'Price', 'Cena za m²': 'PriceSqM', 'Czynsz (dodatkowo)': 'MonthlyRent', 'Finanse': 'Finances', 
'Liczba pokoi': 'Bedrooms', 'Link': 'Link', 'Miejsce': 'Place', 'Oferta od': 'Advert_Owner', 'Powierzchnia': 'SqM',
'Poziom': 'Floor', 'Rodzaj zabudowy': 'BuildingType', 'Rynek': 'Market', 'Umeblowane': 'Furnishings', 'Miasto': 'City'},inplace=True)

In [ ]:
olx_df.head(2)

In [ ]:
olx_df.Advert_Owner.replace(['Osoby prywatnej', 'Biuro / Deweloper'],['Developer','Private'],inplace=True)
olx_df.Advert_Owner.unique()

In [ ]:
olx_df.BuildingType.replace(['Apartamentowiec', 'Blok', 'Dom wolnostojący', 'Kamienica',
                                'Pozostałe', 'Loft', 'Szeregowiec'],
                               ['Apartment', 'Flat', 'House', 'Tenement',
                                'Other', 'Loft', 'Townhouse'],inplace=True)
olx_df.BuildingType.unique()

In [ ]:
olx_df.Market.replace(['Wtórny', 'Pierwotny'],['After', 'Primary'],inplace=True)
olx_df.Market.unique()

In [ ]:
olx_df.head(2)

### Create a column specifying type of offer: For Sale or For Rent:

In [ ]:
olx_df['OfferType'] = np.where(olx_df['MonthlyRent'].isnull(), 'For_Sale', 'For_Rent')

### Create a new dataframe before I start to do more changes

In [ ]:
olx_table = olx_df.copy()

### Remove unnecessary columns with website link and information about loans 

In [ ]:
olx_table.drop(columns={'Finances','Link'}, axis=1, inplace=True)

In [ ]:
olx_table.head(2)

### Create a column with cities using polish characters 

In [ ]:
olx_table['City2'] = olx_table['Place'].str.split(',').str[0]

In [ ]:
olx_table.head(2)

In [ ]:
olx_table.City2.unique()

In [ ]:
# We have some 'no data' records so in that cases let's use city names used in during scraping process (with capitalize func) 
olx_table['City2'] = np.where(olx_table['City2']=='no data',olx_table['City'].str.capitalize(),olx_table['City2'])

In [ ]:
olx_table.City2.unique()

In [ ]:
olx_table.drop(columns={'City'},inplace=True)

In [ ]:
olx_table.rename(columns={'City2':'City'},inplace=True)

### Is there possibility to negotiate a price?

In [ ]:
olx_table['Negotiating'] = np.where(olx_table.Price.str.contains('Do negocjacji'), 1, 0)

In [ ]:
olx_table.head()

### Create a function which remove unnecessary text from chosen column and keep just the first value

But first we need to remove null values from chosen columns

In [ ]:
olx_table.isnull().sum()

In [ ]:
olx_table['PriceSqM'] = olx_table['PriceSqM'].replace(np.nan, '0  zł/m²')

In [ ]:
olx_table['MonthlyRent'] = olx_table['MonthlyRent'].replace(np.nan, '0  zł')

In [ ]:
olx_table.head(2)

In [ ]:
def remove_str(x):
    token1 = x.split(' złDo negocjacji') # Price
    token2 = x.split(' zł') # Price / MonthlyRent
    token3 = x.split(' zł/m²') # PriceSqM
    token4 = x.split(' m²') # SqM
    if len(token1) == 2:
        return (str(token1[0]))
    elif len(token2) == 2:
        return (str(token2[0]))
    elif len(token3) == 2:
        return (str(token3[0]))
    elif len(token4) == 2:
        return (str(token4[0]))
    try:
        return str(x)
    except:
        return False    

In [ ]:
olx_table['Price'] = olx_table['Price'].apply(remove_str)

In [ ]:
olx_table['PriceSqM'] = olx_table['PriceSqM'].apply(remove_str)

In [ ]:
olx_table['SqM'] = olx_table['SqM'].apply(remove_str)

In [ ]:
olx_table['MonthlyRent'] = olx_table['MonthlyRent'].apply(remove_str)

In [ ]:
olx_table.head(2)

### Remove empty spaces in the first two columns

In [ ]:
olx_table.iloc[0,0]

In [ ]:
olx_table['Price'] = olx_table['Price'].str.replace(' ', '')
olx_table['PriceSqM'] = olx_table['PriceSqM'].str.replace(' ', '') 
olx_table.iloc[0,0]

In [ ]:
olx_table.head(2)

8.Change values in Bedrooms column

In [ ]:
olx_table.Bedrooms.unique()

In [ ]:
olx_table.groupby('Bedrooms')['Bedrooms'].count()

In [ ]:
def rooms_number(x):
    if x == 'Kawalerka' or x == '1 pokój':
        return 1
    elif x == '2 pokoje':
        return 2
    elif x == '3 pokoje':
        return 3
    else:
        return 4

In [ ]:
olx_table['Bedrooms'] = olx_table['Bedrooms'].apply(rooms_number)

In [ ]:
olx_table.groupby('Bedrooms')['Bedrooms'].count()

### Keep integer values in the "Floor" and "Furnishings" columns

In [ ]:
olx_table.Floor.unique() # we have some object in here

In [ ]:
olx_table.groupby('Floor')['Floor'].count()

I am not interested in: Attic (Poddasze), floor > 10. So I removed records with these floors and with null. Ground Floor  (Parter) I renamed to 0 

In [ ]:
olx_table.drop(olx_table[(olx_table.Floor=='Poddasze')|(olx_table.Floor=='Powyżej 10')].index, axis=0, inplace=True)

In [ ]:
olx_table.dropna(subset=['Floor'],inplace=True)

In [ ]:
olx_table['Floor'] = olx_table.Floor.replace('Parter','0')

In [ ]:
olx_table['Floor'] = olx_table['Floor'].astype(int)

In [ ]:
olx_table.groupby('Floor')['Floor'].count()

In [ ]:
olx_table['Furnishings'] = np.where(olx_table.Furnishings=='Tak',1,0)

In [ ]:
olx_table.head()

## Divide the olx table to 2 separate tables:

- For Rent
- For Sale

In [ ]:
df_rent = olx_table[olx_table.OfferType=='For_Rent'].copy()
df_sale = olx_table[olx_table.OfferType=='For_Sale'].copy()

### Remove unnecessary columns

In [ ]:
df_sale.head(2)

In [ ]:
df_rent.head(2)

In [ ]:
df_sale.drop(columns={'MonthlyRent','OfferType'}, axis='columns', inplace=True) 

In [ ]:
df_rent.drop(columns={'PriceSqM','Market','OfferType'}, axis='columns', inplace=True)

### Change data types - 'object' to 'float' - for columns with numbers.

In [ ]:
def is_float(x):
    try:
        return float(x)
    except:
        return False

In [ ]:
def RemoveFalse(df,col):
    return df.drop(df[df[col]==False].index,inplace=True)

**For Sale**

In [ ]:
df_sale.dtypes

In [ ]:
df_sale.shape

In [ ]:
df_sale['Price'] = df_sale['Price'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_sale[df_sale.Price==False])} "False" records before remove them \n')
RemoveFalse(df_sale,'Price')
df_sale['Price'] = df_sale['Price'].apply(is_float)
print(f'{len(df_sale[df_sale.Price==False])} "False" records now')

In [ ]:
df_sale['PriceSqM'] = df_sale['PriceSqM'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_sale[df_sale.PriceSqM==False])} "False" records before remove them \n')
RemoveFalse(df_sale,'PriceSqM')
df_sale['PriceSqM'] = df_sale['PriceSqM'].apply(is_float)
print(f'{len(df_sale[df_sale.PriceSqM==False])} "False" records now')

In [ ]:
df_sale['SqM'] = df_sale['SqM'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_sale[df_sale.SqM==False])} "False" records before remove them \n')
RemoveFalse(df_sale,'SqM')
df_sale['SqM'] = df_sale['SqM'].apply(is_float)
print(f'{len(df_sale[df_sale.SqM==False])} "False" records now')

In [ ]:
df_sale.dtypes

In [ ]:
df_sale.shape

In [ ]:
df_sale.head()

In [ ]:
df_sale.describe()

**For Rent**

In [ ]:
df_rent.dtypes

In [ ]:
df_rent.shape

In [ ]:
df_rent['Price'] = df_rent['Price'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_rent[df_rent.Price==False])} "False" records before remove them \n')
RemoveFalse(df_rent,'Price')
df_rent['Price'] = df_rent['Price'].apply(is_float)
print(f'{len(df_rent[df_rent.Price==False])} "False" records now')

In [ ]:
df_rent['MonthlyRent'] = df_rent['MonthlyRent'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_rent[df_rent.MonthlyRent==False])} "False" records before remove them \n')
RemoveFalse(df_rent,'MonthlyRent')
df_rent['MonthlyRent'] = df_rent['MonthlyRent'].apply(is_float)
print(f'{len(df_rent[df_rent.MonthlyRent==False])} "False" records now')

In [ ]:
df_rent['SqM'] = df_rent['SqM'].apply(lambda x: x.replace(',','.')).apply(is_float)
print(f'{len(df_rent[df_rent.SqM==False])} "False" records before remove them \n')
RemoveFalse(df_rent,'SqM')
df_rent['SqM'] = df_rent['SqM'].apply(is_float)
print(f'{len(df_rent[df_rent.SqM==False])} "False" records now')

In [ ]:
df_rent.dtypes

In [ ]:
df_rent.shape

### Create final price for properties 

In [ ]:
df_rent['MonthlyPrice'] = np.where(df_rent.MonthlyRent>1,df_rent.MonthlyRent+df_rent.Price,df_rent.Price)

In [ ]:
df_rent['PriceSqM'] = df_rent.MonthlyPrice/df_rent.SqM

In [ ]:
df_rent.drop(columns={'Price','MonthlyRent'},inplace=True)

In [ ]:
df_rent.head()

In [ ]:
df_rent.describe()

# Next Steps

**Remove too small and too high numbers. Check whish should I drop based on a chart**

1.Histogram (each City)

* SqM: maybe remove small SQM values
* advert per city
* sqm per flat/house
* price for smq per floor

2.plot

* dev/priv


In [ ]:
df_sale.groupby('City')['City'].count().sort_values()

In [ ]:
df_sale.describe()

In [ ]:
df_rent.groupby('City')['City'].count().sort_values()

In [ ]:
df_rent.describe()

# Visualisation
* Where are located our cities
* Where is bigger population

?.Create a list with cities from dataframe with properties for sale

In [ ]:
location = pd.read_csv(location_list,sep=';',encoding='windows-1250')

In [ ]:
location.head(2)

In [ ]:
df_cities = df_sale[['City']].drop_duplicates().reset_index()

In [ ]:
df_cities = df_cities.merge(location,on='City',how='inner')

In [ ]:
city_list =[]
for city in range (len(df_cities)):
    city_list.append(df_cities.iloc[city,1])
    city_list.append(df_cities.iloc[city,2])

In [ ]:
import folium

In [ ]:
map = folium.Map(location=[50.226329, 19.019543],zoom_start=9)

In [ ]:
def add_location(city,lat,lon):
    tooltip = 'Click for more info'
    folium.Marker([lat,lon],
                  popup='<strong>'+city+'</strong>',
                 tooltip=tooltip).add_to(map)

In [ ]:
for x in range(len(city_list)):
    if (x % 2 == 0):
        city = city_list[x]
        loc = city_list[x+1]
        lat = float(loc.strip(',')[1:10])
        lon = float(loc.strip(',')[12:21])
    add_location(city,lat,lon)

In [ ]:
map

### Others

In [ ]:
def plot_bar_x(label1, label2):
    index = np.arange(len(label1))
    plt.bar(index, label2)
    plt.xlabel('', fontsize=10)
    plt.ylabel('Number of offers', fontsize=10)
    plt.xticks(index, label1, fontsize=10, rotation=30)
    plt.title('Amount of offers')
    plt.show()

In [ ]:
a = olx_r.groupby('Owner').agg({'Price': 'count'}).reset_index()
a

In [ ]:
# Create an empty list 
owners_label =[]
amount_label =[]
  
# Iterate over each row 
for index, rows in a.iterrows(): 
    owners_label.append(rows.Owner)
    amount_label.append(rows.Price)
  
print(owners_label,amount_label) 

In [ ]:
plot_bar_x(owners_label, amount_label)

What type of buildings do we have

In [ ]:
building_types = olx_r[olx_r.City=='wodzislaw-slaski'].groupby('Buildings_Type')['Price'].count().reset_index()

building_types_label = []
building_types_amount = []

for index, rows in building_types.iterrows():
    building_types_label.append(rows.Buildings_Type)
    building_types_amount.append(rows.Price)
print(building_types_label,building_types_amount)

In [ ]:
plot_bar_x(building_types_label, building_types_amount)

# Charts... in progress

In [ ]:
wodzislaw = olx_table[olx_table.City=='Wodzislaw-Slaski']

In [ ]:
rybnik = olx_table[olx_table.City=='Rybnik']

In [ ]:
plt.style.use('fivethirtyeight')

x = len(rybnik[rybnik.Offer_Type=='for_sale'])
y = len(rybnik[rybnik.Offer_Type=='for_rent'])
values_a = [x,y]
labels_a = ['For Sale', 'For Rent']
colors_a = ['#174F5F', '#3CAEA3']
explode_a = [0, 0.05]

i = len(wodzislaw[wodzislaw.Offer_Type=='for_sale'])
j = len(wodzislaw[wodzislaw.Offer_Type=='for_rent'])
values_b = [i,j]
labels_b = ['For Sale', 'For Rent']
colors_b = ['#174F5F', '#3CAEA3']
explode_b = [0, 0.05]

# create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2)

# plot each pie chart in a separate subplot
ax1.pie(values_a, labels=labels_a, colors=colors_a, explode=explode_a, startangle=0, autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'})
ax2.pie(values_b, labels=labels_b, colors=colors_b, explode=explode_b, startangle=0, autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'})

ax1.set_title('Rybnik')
ax2.set_title('Wodzislaw')
fig.suptitle('Offert Type Ratio')

#fig.legend(loc='upper center')
plt.tight_layout()
plt.show()